# Custom Loss Functions

To create a Keras model with multiple outputs using the functional API and incorporate custom loss functions, we'll proceed in a few steps. First, we'll define the model architecture with two outputs. Then, we'll implement custom loss functions: Kullback-Leibler (KL) Divergence for output 2 and a custom binary classification loss for output 1. Finally, we'll train the model on dummy data.

Here's how we can do it:

1. Define the Model Architecture: As before, we'll have a model with one input and two outputs.

1. Implement Custom Loss Functions:
        - Custom Binary Classification Loss: A simple example could be a variant of binary cross-entropy.
        - KL Divergence Loss: TensorFlow provides a function for KL Divergence, which we could use directly but in this case we will provide a variation

1. Generate Dummy Data: We'll create dummy data appropriate for our model's input and output specifications.

1. Compile and Train the Model: We'll compile the model with our custom loss functions and then train it.



In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import KLDivergence
import numpy as np

# 1. Model architecture
input_layer = Input(shape=(10,))
dense_layer = Dense(64, activation='relu')(input_layer)
output1 = Dense(1, activation='sigmoid', name='output1')(dense_layer)  # Binary classification output
output2 = Dense(5, activation='softmax', name='output2')(dense_layer)  # Multiclass classification output

model = Model(inputs=input_layer, outputs=[output1, output2])

In [5]:
# 2. Custom binary classification loss
def custom_binary_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    epsilon = 1e-15
    y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
    return -tf.reduce_mean(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))

# 2. Custom KL Divergence Loss
def custom_kl_divergence_loss(y_true, y_pred, scale_factor=1.0):
    kl_loss = tf.keras.losses.KLDivergence()(y_true, y_pred)
    return scale_factor * kl_loss


In [6]:
# 3. Generate dummy data
x_dummy = np.random.random((1000, 10))
y_dummy_output1 = np.random.randint(2, size=(1000, 1))
y_dummy_output2 = np.random.randint(5, size=(1000, 5))


In [7]:
# 4. Compile the model
model.compile(optimizer='adam',
              loss={'output1': custom_binary_loss, 'output2': lambda y_true, y_pred: custom_kl_divergence_loss(y_true, y_pred, scale_factor=2.0)},
              metrics={'output1': ['accuracy'], 'output2': ['accuracy']})

# 4. Train the model
model.fit(x_dummy, {'output1': y_dummy_output1, 'output2': y_dummy_output2}, epochs=10)

Epoch 1/10
32/32 [==============================] - 1s 3ms/step - loss: 13.8724 - output1_loss: 0.7023 - output2_loss: 13.1701 - output1_accuracy: 0.5080 - output2_accuracy: 0.1560
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 13.7080 - output1_loss: 0.6961 - output2_loss: 13.0119 - output1_accuracy: 0.5000 - output2_accuracy: 0.2110
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 13.6947 - output1_loss: 0.6947 - output2_loss: 13.0001 - output1_accuracy: 0.4920 - output2_accuracy: 0.1820
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 13.6861 - output1_loss: 0.6939 - output2_loss: 12.9922 - output1_accuracy: 0.5010 - output2_accuracy: 0.2060
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 13.6787 - output1_loss: 0.6926 - output2_loss: 12.9861 - output1_accuracy: 0.5040 - output2_accuracy: 0.1810
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 13.6744 - output1_loss:

Once a Keras model is trained, you can evaluate its performance on a test dataset and use it to make predictions. Continuing from the previous example, I'll show you how to:

1. Evaluate the Model: We'll evaluate the model on a separate set of dummy data to see how it performs.

2. Use the Model for Prediction: We'll use the model to make predictions based on new input data.

In [10]:
# 1. Evaluate the model

# Generate some dummy test data
x_dummy_test = np.random.random((200, 10))
y_dummy_test_output1 = np.random.randint(2, size=(200, 1))  # Binary labels
y_dummy_test_output2 = np.random.randint(5, size=(200, 5))  # One-hot encoded labels for 5 classes

# Evaluate the model
evaluation = model.evaluate(x_dummy_test, {'output1': y_dummy_test_output1, 'output2': y_dummy_test_output2})
print(f"Test Loss, Test Accuracy for Output 1: {evaluation[1]}, {evaluation[3]}")
print(f"Test Loss, Test Accuracy for Output 2: {evaluation[2]}, {evaluation[4]}")


7/7 [==============================] - 0s 5ms/step - loss: 13.4131 - output1_loss: 0.6929 - output2_loss: 12.7202 - output1_accuracy: 0.5400 - output2_accuracy: 0.2300
Test Loss, Test Accuracy for Output 1: 0.6929206252098083, 0.5400000214576721
Test Loss, Test Accuracy for Output 2: 12.7201509475708, 0.23000000417232513


In [11]:
# 2. Use the model for prediction

# New sample data for prediction
new_sample = np.random.random((1, 10))

# Making predictions
predictions = model.predict(new_sample)
print(f"Predictions for Output 1 (Binary classification): {predictions[0]}")
print(f"Predictions for Output 2 (Multiclass classification): {predictions[1]}")


1/1 [==============================] - 0s 43ms/step
Predictions for Output 1 (Binary classification): [[0.50759894]]
Predictions for Output 2 (Multiclass classification): [[0.20307861 0.2081237  0.20419994 0.17522344 0.2093743 ]]
